In [ ]:
import tifffile
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import tqdm
import joblib as jb
import cv2
from lightgbm import LGBMRegressor
import colorsys

In [5]:
data_path = "C:/Users/ricar/tccpy/data"

In [6]:
b1 = tifffile.TiffFile(data_path + "/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhow_443.tif")
b2 = tifffile.TiffFile(data_path + "/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhow_492.tif")
b3 = tifffile.TiffFile(data_path + "/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhow_560.tif")
b4 = tifffile.TiffFile(data_path + "/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhow_665.tif")
b5 = tifffile.TiffFile(data_path + "/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhow_704.tif")
b6 = tifffile.TiffFile(data_path + "/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhow_740.tif")
b7 = tifffile.TiffFile(data_path + "/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhow_783.tif")
b8 = tifffile.TiffFile(data_path + "/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhow_833.tif")
bat_path = data_path + "/02.Dados_Processados/02.XYZ/BATIMETRIA_PROCESSADA_BABITONGA_2018_DHN_WGS84_UTM22S.xyz"

In [7]:
b2s = tifffile.TiffFile(data_path + "/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhos_492.tif")
b3s = tifffile.TiffFile(data_path + "/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhos_560.tif")
b4s = tifffile.TiffFile(data_path + "/04.Imagens_de_Satelite/02.SR/S2A_MSI_2018_05_20_13_22_41_T22JGR_rhos_665.tif")

In [9]:
b1 = b1.asarray()
b2 = b2.asarray()
b3 = b3.asarray()
b4 = b4.asarray()
b5 = b5.asarray()
b6 = b6.asarray()
b7 = b7.asarray()
b8 = b8.asarray()
b2s = b2s.asarray()
b3s = b3s.asarray()
b4s = b4s.asarray()

AttributeError: 'numpy.ndarray' object has no attribute 'asarray'

In [10]:
def ndwi(first_band, second_band):
    if first_band.shape != second_band.shape:
        return None

    output = np.zeros(first_band.shape)

    for i in tqdm.tqdm(range(first_band.shape[0])):
        for j in range(first_band.shape[1]):
            temp1 = first_band[i][j] - second_band[i][j]
            temp2 = first_band[i][j] + second_band[i][j]    
            output[i][j] = temp1/(temp2 if temp2 != 0 else 1)
                
    return output

In [11]:
ndwib3b8 = ndwi(b3,b8)

100%|██████████████████████████████████████████████████████████████████████████████| 5298/5298 [01:55<00:00, 45.92it/s]


In [12]:
wf_th = 0.3

In [14]:
ndwib3b8 = (ndwib3b8[:,:] > wf_th)
ndwib3b8

array([[False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       ...,
       [False, False, False, ..., False,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ..., False,  True,  True]])

In [2]:
df_data = jb.load('df_bat_data_with_bands.pkl.z')
df_data

,x,y,z,b1,b2,b3,b4,b5,b6,b7,b8
0,1324,1076,5.750000,0.005298,0.010901,0.022501,0.021049,0.018481,0.013979,0.020390,0.016952
1,1324,1077,5.608810,0.005298,0.010777,0.022501,0.021161,0.018481,0.013979,0.020390,0.019481
2,1325,1075,6.010455,0.006965,0.012071,0.024026,0.021244,0.019515,0.009853,0.010949,0.011941
3,1325,1076,5.870476,0.005298,0.009908,0.022134,0.019040,0.018481,0.013979,0.020390,0.011342
4,1326,1074,6.640000,0.007867,0.014787,0.024582,0.021229,0.020979,0.008503,0.013503,0.012797
5,1326,1075,6.396154,0.007867,0.013671,0.023847,0.022346,0.020979,0.008503,0.013503,0.012027
6,1327,1074,6.851875,0.007867,0.013795,0.022621,0.022457,0.020979,0.008503,0.013503,0.011587
7,1328,1073,7.092381,0.008076,0.014545,0.023900,0.023459,0.021205,0.011682,0.016167,0.011167
8,1328,1074,7.087000,0.008006,0.014585,0.023678,0.023125,0.019091,0.012004,0.010785,0.010940
9,1329,1072,7.170000,0.008076,0.015537,0.025002,0.022230,0.021205,0.011682,0.016167,0.010947


In [23]:
lgbm = jb.load('trained_model.pkl.z')

In [19]:
aux = list()
for i in tqdm.tqdm(range(b1.shape[0])):
    for j in range(b1.shape[1]):
        if ndwib3b8[i][j]:
            aux.append(np.array([j,i,b1[i][j],b2[i][j],b3[i][j],b4[i][j],b5[i][j],b6[i][j],b7[i][j],b8[i][j]]))

100%|██████████████████████████████████████████████████████████████████████████████| 5298/5298 [01:11<00:00, 63.51it/s]


In [20]:
dfaux = pd.DataFrame(aux)

In [21]:
dfaux.columns = ["px","py","b1","b2","b3","b4","b5","b6","b7","b8"]

In [24]:
z = lgbm.predict(dfaux.drop(["px","py"],axis=1))

In [25]:
dfaux["z"] = z

In [26]:
dfaux["z"].describe()

count    1.337566e+07
mean     1.753482e+01
std      4.733846e+00
min      9.126111e-02
25%      1.619881e+01
50%      1.869221e+01
75%      2.068725e+01
max      3.090736e+01
Name: z, dtype: float64

In [27]:
dfaux

,px,py,b1,b2,b3,b4,b5,b6,b7,b8,z
0,3214.0,0.0,0.086908,0.079979,0.102661,0.086080,0.056856,0.040526,0.045473,0.053704,2.604056
1,3215.0,0.0,0.086908,0.044002,0.066048,0.049728,0.056856,0.040526,0.045473,0.018538,2.806068
2,3216.0,0.0,0.037320,0.041214,0.059458,0.035885,0.027154,0.015188,0.014515,0.012565,5.751935
3,3217.0,0.0,0.037320,0.038629,0.057144,0.034436,0.027154,0.015188,0.014515,0.007505,7.489255
4,3218.0,0.0,0.037875,0.040047,0.057573,0.031751,0.023831,0.009670,0.009858,0.009758,14.969525
5,3219.0,0.0,0.037875,0.040909,0.058304,0.032085,0.023831,0.009670,0.009858,0.008328,14.931090
6,3220.0,0.0,0.038500,0.040811,0.057613,0.032633,0.025956,0.010154,0.009950,0.009598,13.825122
7,3221.0,0.0,0.038500,0.039580,0.057126,0.032299,0.025956,0.010154,0.009950,0.009488,13.956033
8,3222.0,0.0,0.029588,0.039932,0.056463,0.032471,0.025252,0.010358,0.009122,0.009260,16.475637
9,3223.0,0.0,0.029588,0.039562,0.057560,0.030909,0.025252,0.010358,0.009122,0.009260,15.447516


In [ ]:
jb.dump(dfaux,"prediction_data.pkl.z")